## End to end model:feature extraction

###  1 import libs

In [1]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras import optimizers
from keras.applications.vgg19 import preprocess_input
from keras.utils.training_utils import multi_gpu_model
import tensorflow as tf

Using TensorFlow backend.


## 2 build pretrained model¶

In [2]:
with tf.device('/cpu:0'):
    input_tensor = Input((224, 224, 3))
    x = Lambda(vgg19.preprocess_input)(input_tensor)

    base_model = VGG19(input_tensor=x, weights='imagenet', include_top=False)
    base_model.summary()

    print("###body###")

    m_out = base_model.output

    flatten = Flatten(name='flatten')(m_out)

    fc1 = Dense(4096, activation='relu', name='fc1')(flatten)
    drop_fc1 = Dropout(0.5, name='drop_fc1')(fc1)

    fc2 = Dense(512, activation='relu', name='fc2')(drop_fc1)
    drop_fc2 = Dropout(0.5, name='drop_fc2')(fc2)

    predictions = Dense(2, activation='softmax', name='predictions')(drop_fc2)

    for layer in base_model.layers:  
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)
    model.summary()
    print("###whole###")

    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

parallel_model = multi_gpu_model(model, gpus=4)
parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
parallel_model.summary()
print("###parallel###")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________

### 3 build param

In [3]:
batch_size = 128
epochs = 5

total_imgs_num = 27140 + 3023
train_imgs_num = 27140
valid_imgs_num = 3023

img_size = (224, 224)

### 4 build data & first train

In [ ]:
img_gen_t = ImageDataGenerator()

train_generator = img_gen_t.flow_from_directory("/home/tsimage/high_speed_data/tct_data_samesize_0718_224/train", 
                                                img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("/home/tsimage/high_speed_data/tct_data_samesize_0718_224/valid",
                                                img_size, shuffle=True, batch_size=batch_size)

parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=train_imgs_num//batch_size, 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=valid_imgs_num//batch_size)

model.save_weights("vgg19_first_train.h5")

Found 27140 images belonging to 2 classes.
Found 3023 images belonging to 2 classes.
Epoch 1/5
212/212 [==============================] - 212s 999ms/step - loss: 5.3770 - acc: 0.6654 - val_loss: 5.3599 - val_acc: 0.6675
Epoch 2/5


### 5 build finetune model

In [5]:
with tf.device('/cpu:0'):
    input_tensor = Input((224, 224, 3))
    x = Lambda(vgg19.preprocess_input)(input_tensor)

    base_model = VGG19(input_tensor=x, weights='imagenet', include_top=False)
    base_model.summary()

    print("###body###")

    m_out = base_model.output

    flatten = Flatten(name='flatten')(m_out)

    fc1 = Dense(4096, activation='relu', name='fc1')(flatten)
    drop_fc1 = Dropout(0.5, name='drop_fc1')(fc1)

    fc2 = Dense(512, activation='relu', name='fc2')(drop_fc1)
    drop_fc2 = Dropout(0.5, name='drop_fc2')(fc2)

    predictions = Dense(2, activation='softmax', name='predictions')(drop_fc2)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.load_weights('vgg19_first_train.h5')
    model.summary()
    print("###whole###")

    sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

parallel_model = multi_gpu_model(model, gpus=4)
parallel_model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
parallel_model.summary()
print("###parallel###")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
lambda_6 (Lambda)            (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
__________

### 6 build param

In [6]:
batch_size = 128
epochs = 100

total_imgs_num = 27140 + 3023
train_imgs_num = 27140
valid_imgs_num = 3023

img_size = (224, 224)

### 7 build data & finetune train

In [7]:
img_gen_t = ImageDataGenerator(rotation_range=180,
                               width_shift_range=0.3,
                               height_shift_range=0.3,
                               zoom_range=0.3,
                               horizontal_flip=True,
                               vertical_flip=True)

train_generator = img_gen_t.flow_from_directory("/home/tsimage/high_speed_data/tct_data_samesize_0718_224/train", 
                                                img_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator()
valid_generator = img_gen_v.flow_from_directory("/home/tsimage/high_speed_data/tct_data_samesize_0718_224/valid", 
                                                img_size, shuffle=True, batch_size=batch_size)


parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=train_imgs_num//batch_size, 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=valid_imgs_num//batch_size)

model.save_weights("vgg19_finetune.h5")

Found 27140 images belonging to 2 classes.
Found 3023 images belonging to 2 classes.
Epoch 1/100
212/212 [==============================] - 231s 1s/step - loss: 0.4616 - acc: 0.7783 - val_loss: 0.3607 - val_acc: 0.8336
Epoch 2/100
212/212 [==============================] - 230s 1s/step - loss: 0.4304 - acc: 0.7961 - val_loss: 0.3182 - val_acc: 0.8536
Epoch 3/100
212/212 [==============================] - 229s 1s/step - loss: 0.3542 - acc: 0.8364 - val_loss: 0.3130 - val_acc: 0.8512
Epoch 4/100
212/212 [==============================] - 229s 1s/step - loss: 0.3317 - acc: 0.8511 - val_loss: 0.2614 - val_acc: 0.8947
Epoch 5/100
212/212 [==============================] - 229s 1s/step - loss: 0.3321 - acc: 0.8515 - val_loss: 0.3174 - val_acc: 0.8556
Epoch 6/100
212/212 [==============================] - 228s 1s/step - loss: 0.3092 - acc: 0.8626 - val_loss: 0.2270 - val_acc: 0.9059
Epoch 7/100
212/212 [==============================] - 228s 1s/step - loss: 0.2958 - acc: 0.8681 - val_loss: 0.